# Tutorial 1

**In this tutorial you will:**
- learn the Flow basics and implement your first Flow [Section 1](#1-writing-your-first-atomicflow-reversenumberatomicflow)
- understand the basics of Flows engine and serve your first Flow [Section 2](#2-flow-engine)
- learn how to call served Flows [Section 2](#24-getting-an-instance-of-the-served-flow-via-a-proxy)
- learn how to compose Flows into more complex Flows [Section 3](#3writing-your-first-compositeflow-reversenumbersequentialflow)

In [1]:
%load_ext autoreload
%autoreload 2
#imports
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.utils import serve_utils
from aiflows.utils import colink_utils
from aiflows.workers import run_dispatch_worker_thread
from aiflows.base_flows import AtomicFlow
from aiflows.messages import FlowMessage
import sys
import os
sys.path.append("..")
from utils import compile_and_writefile, dict_to_yaml

#Specify path of your flow modules
FLOW_MODULES_PATH = "./"

/Users/nicolasbaldwin/opt/miniconda3/envs/mockenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Writing your First AtomicFlow: ReverseNumberAtomicFlow

In this section we will introduce the concept of a Flow and implement a simple Flow that reverses a number passed in the input message.

In a nutshell, Flows are objects that perform semantically meaningful unit of work and comunicate via messages.

There are two types of Flows: 
1) **Atomic Flows:** Performs the computation directly
2) **Composite Flows:** Orchestrates other Flows in performing the computation

When a message is send to a Flow, the Flow processes the message according to the logic specified in its `run` method. We will now go through the implementation of the `ReverseNumberAtomicFlow`: a flow who given a number, reverses it (e.g., 12345-> 54321).

In Section 3, we see provide an example implementation of a Composite Flow.

### 1.1 Writing the ReverseNumberAtomicFlow

To implement a Flow, you must define its run method. This method takes an input_message as its input, which is an instance of the FlowMessage class. Within the FlowMessage, there are two important attributes to consider:

 - 1. `data`: This attribute is a dictionary containing the data that the Flow will process.
 - 2. `reply_data`: Another dictionary, it holds necessary information to send a reply message back to the sender of the input message. Typically, direct modification of this attribute isn't recommended. Instead, utilize the provided methods for modifying it (these methods will be introduced throughout the notebook).

When defining a Flow, it's crucial to specify the `input_interface` and `output_interface` in the configuration file. These interfaces specify the expected keys in the `data` attribute of the `input_message` and what will be returned in the `data` attribute when sending the message back. This ensures proper calling of your flow by other users. For instance, in the `ReverseNumberAtomicFlow`, the `input_interface` expects a key `number` in the `data` attribute of the `input_message`, while the `output_interface` will include a key `reversed_number` in the `data` attribute of the sent message. Once the number is reversed, package the response dictionary into a message using the `package_output_message` method. This method packages the dictionary into a message and adjusts the `reply_data` attribute of the message to match that of the input message, ensuring it is sent back to the original sender. Finally, the reply is sent back using the `send_message` method.


In [2]:
%%compile_and_writefile ./ReverseNumberFlowModule/ReverseNumberAtomicFlow.py


from aiflows.base_flows import AtomicFlow
from aiflows.messages import FlowMessage

class ReverseNumberAtomicFlow(AtomicFlow):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    # Customize the logic within this function as needed for your specific flow requirements.
    def run(self, input_message: FlowMessage):
        ## ~~~~ Getting the input ~~~~

        # Get the input dictionary from the input message
        input_data = input_message.data
        
        #get input number from data dictionary
        input_number = input_data["number"]
        
        ## ~~~~ Main logic ~~~~
        ########## YOUR CODE GOES HERE ##########
        # TODO: Reverse the input number (e.g. 1234 -> 4321)
        reversed_number = int(str(input_number)[::-1])
        ########## YOUR CODE GOES HERE ##########
        
        # ~~~ Preparing the response message ~~~
        response = {"reversed_number": reversed_number}
        
        # This method packages the `response` in a FlowMessage object 
        # containing the necessary metadata to send the message back
        # to the sender of the input message  
        reply = self.package_output_message(
            input_message=input_message,
            response=response,
        )
        
        # ~~~ Sending the response ~~~
        self.send_message(
            reply
        )



Note: Please be aware that the cell magic `compile_and_writefile` is saving the cell above at `./ReverseNumberFlowModule/ReverseNumberAtomicFlow.py`.



### 1.2 Creating the Default Configuration File for the Reverse Number Flow Class

For each Flow class, there exists a corresponding default configuration file. This file delineates the parameters to be configured for the Flow. Access to each field in the configuration file is facilitated within the Flow class through the syntax `self.flow_config["FIELD_NAME"]`. For instance, to retrieve the `input_interface` field, you would utilize `self.flow_config["input_interface"]`.

Now, let's proceed with composing the configuration file for the `ReverseNumberAtomicFlow`. It's noteworthy that we define it as a dictionary here and subsequently save it to a `.yaml` file. However, you also have the option to directly craft the configuration within a `.yaml` file. 

**IMPORTANT: Ensure that the configuration file is named identical to the Flow class name and is placed in the same directory as the flow. If you don't, it won't be considered as the default configuration file of that flow.**

In [3]:
default_config_reverse_number = \
{
    "name": "ReverseNumber",
    "description": "A flow that takes in a number and reverses it.",

    "_target_": "ReverseNumberFlowModule.ReverseNumberAtomicFlow.ReverseNumberAtomicFlow.instantiate_from_default_config",

    "input_interface": "number",
    "output_interface": "reversed_number",
}
dict_to_yaml(default_config_reverse_number, "./ReverseNumberFlowModule/ReverseNumberAtomicFlow.yaml")

Let's dissect the configuration file for the `ReverseNumberAtomicFlow`:

- **`name` and `description`**: These parameters are straightforward. When defining a Flow, these must always be specified. Although they don't directly affect the functionality of the Flow, they are essential for documentation purposes.

- **`_target_`**: This parameter specifies how the flow will be instantiated by `hydra`, a framework for configuring complex applications. It should always specify the path to the Flow class followed by `instantiate_from_default_config`. This standardizes the instantiation process from the default configuration file. For example, the value of `_target_` is `ReverseNumberFlowModule.ReverseNumberAtomic.ReverseNumberAtomicFlow.instantiate_from_default_config`, where `ReverseNumberFlowModule/ReverseNumberAtomic.py` is the path to the Flow class, and the class name in `ReverseNumberAtomic.py` is `ReverseNumberAtomic`.

- **`input_interface`**: This parameter outlines the keys expected in the `data` attribute of the input message. Here, the `number` key is expected.

- **`output_interface`**: Specifies the keys present in the `data` attribute of the message sent back. In this case, the `reversed_number` key will be included.

## 2. Flow Engine

In this section we introduce the basics of the Flows Engine, and guide you through the process of serving and accessing served Flows.

The Flows Engine is build on top of [CoLink](https://github.com/CoLearn-Dev/colink-sdk-python-dev). As mentioned in their documentation:

> CoLink provides a unified interface for the user, storage, communication, and computation. Extending gRPC, CoLink simplifies the development of multi-party protocols and allow implementations in different programming languages to work together consistently. With a unified interface that increases potential data contributors, CoLink has the potential to enable larger-scale decentralized data collaboration and unlock the true value of data.

In simpler terms, CoLink allows aiflows to run multiple flows, which can be spread across multiple machines to complete a task. It enables communication between flows hosted on different machines. Users can serve their flows, and others can access them by obtaining an instance of the served flow and interacting with it through a proxy.

In the following sections, we will guide you through:
- Connecting to an existing colink server or starting a local one
- Serving a flow
- Starting a worker to run flows
- Getting an instance of the served flow via a proxy
- Calling the flow through the proxy

### 2.1 Connecting to an Existing Colink Server or Starting a Local One

You have the option to either connect to an existing CoLink server or start a local one. Local servers are particularly useful for testing and development purposes. On the other hand, connecting to an existing server is beneficial when you need to access flows served by others or serve your own flows to others.

For this AMLD workshop, we have initiated a CoLink server for you to connect to, allowing you to serve your flows and access flows served by others. However, for the purpose of this tutorial, we will guide you through starting a local server. Instructions for connecting to an existing server will be provided later during the workshop.

To start a local server run the cell below (the server will run as long as your kernel is running).

In [4]:
#Start the colink server and creates a user (you)
#Returns the colink object who contains the information of the user and the server
cl = colink_utils.start_colink_server()

### 2.2 Serving A Flow

To serve a flow, you need to define its class and `run` method, along with its default configuration file (as we've done in the previous section for the `ReverseNumberAtomicFlow`). Once the flow is defined, you can serve it using the `serve_flow` function. Essentially, this function saves the flow's default configuration in the CoLink server's storage to be able to instantiate instances of it when requested.

The key parameters of the `serve_flow` function are as follows:
- `cl`: The CoLink object containing user and server information (this object is created when you connect to a server).
- `flow_class_name`: Specifies the class of the flow to be served (path to the class + class name).
- `flow_endpoint`: The endpoint where the flow will be served. This serves as a unique identifier for accessing the flow. If the endpoint is already in use by another flow, your flow will not be served.
- `singleton`: (Default is False) If set to True, only one instance of the flow will be returned when a user requests it. In other words, multiple users can "talk" to the same flow (e.g., all user "talks" to the same chatbot). If set to False, each user will get a different flow instance (e.g., each user "talks"  to a different chatbots but of the same class).
- `parallel_dispatch`: (Default is False) If set to True, the flow can handle multiple requests simultaneously. If set to False, the flow can only handle one request at a time. Note that this makes the flow stateless.

For example, in the cell below, we serve the `ReverseNumberAtomicFlow` at the endpoint "reverse_number" as a singleton.




In [5]:
#serve the 
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="ReverseNumberFlowModule.ReverseNumberAtomicFlow.ReverseNumberAtomicFlow",
    flow_endpoint="reverse_number",
    singleton=True,
)

[2024-03-22 00:59:37,214][aiflows.utils.serve_utils:116][INFO] - Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow.ReverseNumberAtomicFlow at flows:reverse_number.
[2024-03-22 00:59:37,217][aiflows.utils.serve_utils:117][INFO] - dispatch_point: coflows_dispatch
[2024-03-22 00:59:37,219][aiflows.utils.serve_utils:118][INFO] - parallel_dispatch: False
[2024-03-22 00:59:37,310][aiflows.utils.serve_utils:119][INFO] - singleton: True



True

### 2.3 Starting a Worker to Run Flows

As a server of the flow, you must be the one to execute it when called. To execute a flow, you need to start a worker. The worker is responsible for running the flow's logic and will be activated whenever there are new messages to process. It's important to note that a single worker can handle multiple flows; they are not limited to a single flow exclusively.

To start a worker, run the cell here below:

In [6]:
# Start a worker thread to handle incoming messages
run_dispatch_worker_thread(cl)

[2024-03-22 00:59:37,764][aiflows.workers.dispatch_worker:220][INFO] - Dispatch worker started in attached thread.
[2024-03-22 00:59:37,817][aiflows.workers.dispatch_worker:221][INFO] - dispatch_point: coflows_dispatch


### 2.4 Getting an Instance of the Served Flow via a Proxy

Now that the flow is served, you can access it by obtaining an instance of the served flow through a proxy. The proxy is an `AtomicFlow` object that allows you to send messages to the requested flow instance and receive responses from it. To obtain an instance of the served flow, you can use the `get_flow_instance` function.

The main parameters of the `get_flow_instance` function are:

- `cl`: The CoLink object.
- `flow_endpoint`: The endpoint where the flow is served (this should match the endpoint used to serve the flow).
- `user_id`: The user ID of the user serving the flow. Since we're serving a local server, you can simply set it to "local".
- `config_overrides`: (Default is None) Any configuration overrides to be applied to the flow. This is useful when you want to modify the default configuration of the flow. For example, you might want to change the language of a chatbot. The configuration overrides are merged with the default configuration of the flow, with the overrides taking precedence.

In [7]:
# Get an instance of the flow
proxy_reverse_number_flow = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="reverse_number", ### TODO - SPECIFY THE CORRECT FLOW ENDPOINT
    user_id="local",
)

[2024-03-22 00:59:38,294][aiflows.utils.serve_utils:336][INFO] - Mounted 40912248-9a21-4766-9c71-f145e3dd78d0 at flows:reverse_number:mounts:local:40912248-9a21-4766-9c71-f145e3dd78d0


### 2.5. Calling the Reverse Number Flow via the Proxy

Now that you have a proxy to the served flow, you can send messages to it and receive responses. To send a message to the flow, you need to package your data dictionary with the `package_input_message` method and then send a message via the proxy using the `get_reply_future` method. `get_reply_future` sends a message to the flow with the data dictionary and the necessary information (in the `reply_data` attribute of the message) to get a reply back. It returns a future object that you can use to retrieve the reply from the flow. The message can be retrieved from the future object using either the `get_data` method (which returns the `data` attribute of the reply message) or the `get_message` method. Both of these methods will block until the reply is received.

Without further ado, let's send a message to the `ReverseNumberAtomicFlow` to reverse the number 12345!


In [8]:
input_data = {"id": 0, "number": 12345}

# Package your data in a Flow Message

## Option 1: Via the FlowMessage class
# input_message = FlowMessage(
#     data=input_data,
# )

## Option 2 (prefered): Via the package input message method
input_message = proxy_reverse_number_flow.package_input_message(input_data)

# Send a message to reverse number and ask to get an answer back in a future

future = proxy_reverse_number_flow.get_reply_future(input_message)

# Get the response from the future
#To get the response as a data dictionary
reply_data = future.get_data()
#To get the response as a FlowMessage object
reply_message = future.get_message()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")



[2024-03-22 00:59:38,586][aiflows.workers.dispatch_worker:113][INFO] - 
~~~ Dispatch task ~~~
[2024-03-22 00:59:38,591][aiflows.workers.dispatch_worker:155][INFO] - flow_endpoint: reverse_number
[2024-03-22 00:59:38,592][aiflows.workers.dispatch_worker:156][INFO] - flow_id: 40912248-9a21-4766-9c71-f145e3dd78d0
[2024-03-22 00:59:38,592][aiflows.workers.dispatch_worker:157][INFO] - owner_id: local
[2024-03-22 00:59:38,593][aiflows.workers.dispatch_worker:158][INFO] - message_paths: ['push_tasks:2a04b973-9326-470a-859b-c05dc2b0b4a5:msg']
[2024-03-22 00:59:38,595][aiflows.workers.dispatch_worker:159][INFO] - parallel_dispatch: False

[2024-03-22 00:59:38,688][aiflows.workers.dispatch_worker:182][INFO] - Input message source: Proxy_reverse_number
[2024-03-22 00:59:39,955][aiflows.workers.dispatch_worker:113][INFO] - 
~~~ Dispatch task ~~~
[2024-03-22 00:59:39,966][aiflows.workers.dispatch_worker:155][INFO] - flow_endpoint: ReverseNumberSequentialFlowBlocking
[2024-03-22 00:59:39,967][aiflow

Data sent:
 {'id': 0, 'number': 12345} 

REPLY:
 {'reversed_number': 54321} 



## 3.Writing your First CompositeFlow: ReverseNumberSequentialFlow

In this section we will write a simple Composite Flow that reverses a number passed in the input message twice (e.g., 12345-> 54321 -> 12345).

As mentioned in section 1, **Composite Flows:** are used to orchestrate other Flows in performing the computation. Just like **Atomic Flows**, Composite Flows have a `run` method that takes an input message as its input. The `run` method of a Composite Flow typically calls other flows via their proxies, processes the responses, and sends a reply back. The flows that a Composite Flow orchestrates are referred to as its subflows.


### 3.1 Writing the ReverseNumberSequentialFlow Flow Class

#### **Key Interfaces**

The `KeyInterface` class is helpful when you need to apply transformations to the data dictionary when passing from one flow to another and to pass a dictionary that satisfies the `input_interface` of the flow you wish to call. It allows you to define these transformations in a structured way. While you can apply these transformations directly in the `run` method of the Composite Flow, the `KeyInterface` class provides a more organized approach. The `KeyInterface` class takes the following input parameters (which define the transformations) and executes them in the order they are defined:
- `keys_to_rename`: A dictionary where the keys are the keys to rename and the values are the new names of the keys. For example, `{"old_key": "new_key"}` transforms `{"old_key": 1}` to `{"new_key": 1}`.

- `keys_to_copy`: A dictionary where the keys are the keys to copy and the values are the new names of the keys. For example, `{"key_a": "key_b"}` transforms `{"key_a": 1}` to `{"key_a": 1, "key_b": 1}`.
- `keys_to_set`: A dictionary where the keys are the keys to set and the values are the values to set. For example, `{"key_a": 1}` transforms `{}` to `{"key_a": 1}`.

- `additional_transformations`: A list of Callables where you can define custom transformations. Each Callable should take the arguments `data_dict` and return a dictionary as output.For example:
    ```python
        def increment_key_a(data_dict, **kwargs): 
            return {**input_data,**{"key_a": input_data["key_a"] + 1}}
    ```

- `key_to_select`: A list of keys to select from the dictionary. For example, `["key_a"]` transforms `{"key_a": 1, "key_b": 2}` to `{"key_a": 1}`.

- `key_to_delete`: A list of keys to delete from the dictionary. For example, `["key_a"]` transforms `{"key_a": 1, "key_b": 2}` to `{"key_b": 2}`.

#### **Blocking vs Non-Blocking Calls**

There are two ways to call a subflow from a Composite Flow via its proxy:
1. **Blocking Calls**: The Composite Flow sends a message to the subflow and waits for the response before proceeding (and therefore stays in the `run` method until the response is received). Blocking calls are made using the `get_reply_future` method we introduced in the previous section (`get_reply_future` return a future object that you can use to retrieve the reply from the flow via `future.get_data()` or `future.get_message()`, these "get" methods are what are blocking).
2. **Non-Blocking Calls**: The Composite Flow sends a message to the subflow and proceeds without waiting for the response. The response is expected to be sent in the input message queue of the Composite Flow. Therefore, you must exit the `run` method and the flow will be called again when the response is received. Non-blocking calls are made using the `send_message` method.

##### Pros and cons of Blocking vs Non-Blocking Call? 

- **Blocking calls** might be more intuitive to use and understand, but they are less efficient because the Composite Flow is blocked while waiting for the response. Meaning that you are blocking one worker for the duration of the call. Therefore, you must have at least 2 worker threads running to be able to make a blocking call to a subflow and still be able to process other messages.

- **Non-blocking calls** are more efficient because the Composite Flow is not blocked while waiting for the response (one worker can run the whole flow). However, they are more complex to use because you must manage the state of the flow and the responses.

In the end, the choice between blocking and non-blocking calls depends on the specific use case, the requirements of the flow and your preference.

Here below we showcase the implementation of the `ReverseNumberSequentialFlow` class with both blocking and non-blocking calls to the `ReverseNumberAtomicFlow`.


#### 3.1.1 ReverseNumberSequential with Blocking Calls

In [9]:
%%compile_and_writefile ReverseNumberFlowModule/ReverseNumberSequentialFlowBlocking.py


from aiflows.base_flows import CompositeFlow
from aiflows.messages import FlowMessage
from aiflows.interfaces import KeyInterface
class ReverseNumberSequentialFlowBlocking(CompositeFlow):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        #~~~~~~~~~~~ Key Transformation solution 1 ~~~~~~~~~~~
        self.input_interface_second_reverse_flow = KeyInterface(
            keys_to_rename= {"reversed_number": "number"},
            keys_to_select= ["number"],
        )
    
        self.ouput_interface_reply = KeyInterface(
            keys_to_rename= {"reversed_number": "output_number"},
            keys_to_select = ["output_number"],
        )
                
    # Customize the logic within this function as needed for your specific flow requirements.
    def run(self, input_message: FlowMessage):
        
        #call first reverse number
        future_first_reverse = self.subflows["first_reverse_flow"].get_reply_future(
                input_message
            )
        #get response from first reverse number
        first_reverse_response = future_first_reverse.get_message()
        
        #prepare response for the second reverse number
        first_reverse_response = self.input_interface_second_reverse_flow(first_reverse_response) 
        
        #call second reverse number:
        future_second_reverse = self.subflows["second_reverse_flow"].get_reply_future(
            first_reverse_response
        )
        
        second_reverse_response = future_second_reverse.get_data()
        
        #can call key interface tranformation on both message and dictionaries
        #prepare response for the initial message
        response = self.ouput_interface_reply(second_reverse_response)
        
        reply = self.package_output_message(
            input_message = input_message,
            response = response
        )
        
        self.send_message(
            reply
        )
        

In [10]:
default_config_reverse_number_sequential_blocking = \
{
    "name": "ReverseNumberTwiceBlocking",
    "description": "A sequential flow that reverses a number twice.",

    # TODO: Define the target
    "_target_": \
        "ReverseNumberFlowModule.ReverseNumberSequentialFlowBlocking.ReverseNumberSequentialFlowBlocking.instantiate_from_default_config",

    "input_interface": "number",
    "output_interface": "output_number",
    
    "subflows_config": {
        "first_reverse_flow": {
            "_target_": "aiflows.base_flows.AtomicFlow.instantiate_from_default_config",
            "user_id": "local",
            "flow_endpoint": "reverse_number",
            "name": "A proxy flow that calls reverse number to reverse number AGAIN.",
            "description": "A proxy flow that calls reverse number to reverse number.",
        },
        "second_reverse_flow": {
            "_target_": "aiflows.base_flows.AtomicFlow.instantiate_from_default_config",
            "user_id": "local",
            "flow_endpoint": "reverse_number",
            "name": "Proxy Second Reverse",
            "description": "A proxy flow that calls reverse number to reverse number AGAIN.",
        },
    }
}

Let's dissect the new configuration file for the `ReverseNumberSequentialFlowBlocking`:

- **`name` and `description`, `_target` , `input_interface`, `ouput_interface`**: These parameters are described in the previous section.

- **`subflows_config`**:  Each subflow is articulated as a key-item pair within a dictionary. The key denotes the name assigned to the subflow, while the corresponding item is a dictionary encapsulating the configuration of the subflow. In this instance, subflows are outlined with their default configuration, incorporating overrides for the name and description of each flow. Noteworthy keys:
  - `flow_endpoint` and `user_id` match the `reverse_number` endpoint and `local` user ID, respectively, which were used to serve the `ReverseNumberAtomicFlow`. This allows us to get an instance of the flow we served earlier when calling `get_flow_instance` (we don't have to serve `ReverseNumberAtomicFlow` again).
  - `_target_`: The target here is `aiflows.base_flows.AtomicFlow.instantiate_from_default_config`, which is an atomic flow. Remember that proxy flows are atomic flows, so we can use the same target for all of them. Note that you can also use the target of the flow you are calling, but this is not necessary (also, when you are calling a flow served by some other user you might not have access to the target of the flow, so you can use the target of the atomic flow).

In [11]:
#Serve the flow (just like before)
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="ReverseNumberFlowModule.ReverseNumberSequentialFlowBlocking.ReverseNumberSequentialFlowBlocking",
    flow_endpoint="ReverseNumberSequentialFlowBlocking",
)

[2024-03-22 00:59:39,306][aiflows.utils.serve_utils:116][INFO] - Started serving ReverseNumberFlowModule.ReverseNumberSequentialFlowBlocking.ReverseNumberSequentialFlowBlocking at flows:ReverseNumberSequentialFlowBlocking.
[2024-03-22 00:59:39,308][aiflows.utils.serve_utils:117][INFO] - dispatch_point: coflows_dispatch
[2024-03-22 00:59:39,310][aiflows.utils.serve_utils:118][INFO] - parallel_dispatch: False
[2024-03-22 00:59:39,312][aiflows.utils.serve_utils:119][INFO] - singleton: False



True

In [12]:
# Start a second worker thread (since the flow uses blocking calls, 2 workers are needed)
run_dispatch_worker_thread(cl)

[2024-03-22 00:59:39,445][aiflows.workers.dispatch_worker:220][INFO] - Dispatch worker started in attached thread.
[2024-03-22 00:59:39,449][aiflows.workers.dispatch_worker:221][INFO] - dispatch_point: coflows_dispatch


In [13]:
# Get an instance of the flow and run it !!
proxy_reverse_sequential_blocking = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="ReverseNumberSequentialFlowBlocking", #TODO: SPECIFY THE ENDPOINT
    user_id="local",
    config_overrides=default_config_reverse_number_sequential_blocking,
)

input_data = {"id": 0, "number": 12345}

# Package your data in a Flow Message
## Via the package input message method
input_message = proxy_reverse_sequential_blocking.package_input_message(input_data)

# Send a message to reverse number and ask to get an answer back in a future

future = proxy_reverse_sequential_blocking.get_reply_future(input_message)

# Get the response from the future
#To get the response as a data dictionary
reply_data = future.get_data()
#To get the response as a FlowMessage object
reply_message = future.get_message()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")

[2024-03-22 00:59:39,806][aiflows.utils.serve_utils:543][INFO] - Fetched singleton 40912248-9a21-4766-9c71-f145e3dd78d0
[2024-03-22 00:59:39,879][aiflows.utils.serve_utils:543][INFO] - Fetched singleton 40912248-9a21-4766-9c71-f145e3dd78d0
[2024-03-22 00:59:39,916][aiflows.utils.serve_utils:336][INFO] - Mounted bba265c4-ba16-43dc-9eb3-98cfc2eebdfd at flows:ReverseNumberSequentialFlowBlocking:mounts:local:bba265c4-ba16-43dc-9eb3-98cfc2eebdfd
Data sent:
 {'id': 0, 'number': 12345} 

REPLY:
 {'output_number': 12345} 



[2024-03-22 00:59:42,750][aiflows.workers.dispatch_worker:113][INFO] - 
~~~ Dispatch task ~~~
[2024-03-22 00:59:42,780][aiflows.workers.dispatch_worker:155][INFO] - flow_endpoint: reverse_number
[2024-03-22 00:59:42,781][aiflows.workers.dispatch_worker:156][INFO] - flow_id: 40912248-9a21-4766-9c71-f145e3dd78d0
[2024-03-22 00:59:42,783][aiflows.workers.dispatch_worker:157][INFO] - owner_id: local
[2024-03-22 00:59:42,784][aiflows.workers.dispatch_worker:158][INFO] - message_paths: ['push_tasks:ebe36ca9-58bd-49de-9108-34b28e0bb11c:msg']
[2024-03-22 00:59:42,785][aiflows.workers.dispatch_worker:159][INFO] - parallel_dispatch: False

[2024-03-22 00:59:43,355][aiflows.workers.dispatch_worker:182][INFO] - Input message source: first_reverse_flow
[2024-03-22 00:59:43,475][aiflows.workers.dispatch_worker:113][INFO] - 
~~~ Dispatch task ~~~
[2024-03-22 00:59:43,488][aiflows.workers.dispatch_worker:155][INFO] - flow_endpoint: reverse_number
[2024-03-22 00:59:43,514][aiflows.workers.dispatch_work

#### 3.1.1 ReverseNumberSequential with Non-Blocking Calls

Note: Any data that you must save from one `run` method pass to another must be saved in the `self.flow_state` attribute of the flow. This attribute is a dictionary that persists between calls to the `run` method. You can use it to save any data you need to keep between calls.

In [14]:
%%compile_and_writefile ReverseNumberFlowModule/ReverseNumberSequentialFlowNonBlocking.py


from aiflows.base_flows import CompositeFlow
from aiflows.messages import FlowMessage
from aiflows.interfaces import KeyInterface
class ReverseNumberSequentialFlowNonBlocking(CompositeFlow):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        #~~~~~~~~~~~ Key Transformation solution 1 ~~~~~~~~~~~
        self.input_interface_second_reverse_flow = KeyInterface(
            keys_to_rename= {"reversed_number": "number"},
            keys_to_select= ["number"],
        )
    
        self.ouput_interface_reply = KeyInterface(
            keys_to_rename= {"reversed_number": "output_number"},
            keys_to_select = ["output_number"],
        )
        
        self.next_state_dict = {
            "first_reverse_flow": "second_reverse_flow",
            "second_reverse_flow": "reply_to_message",
            "reply_to_message": "first_reverse_flow"
        }
    def set_up_flow_state(self):
        super().set_up_flow_state()
        self.flow_state = {"current_state": "first_reverse_flow"}
        
    def get_next_state(self):
        return self.next_state_dict[self.flow_state["current_state"]]
        
    # Customize the logic within this function as needed for your specific flow requirements.
    def run(self, input_message: FlowMessage):
        
        #Run here is a bit like a switch statement where we decide which flow to call next.
        # We then call the next flow and pass the input message to it. which we expect to get a reply from 
        # back in the input queue (which will call the run method againg)
        current_state = self.flow_state["current_state"]
        
        #Case where we need to reverse the number for the first time
        if current_state == "first_reverse_flow":
            #Save the initial message to the state
            self.flow_state["initial_message"] = input_message
            
            #Prepares message by editing `reply_data` so that it can get the reply from the first flow
            #back in the input queue of the composite flow 
            message_for_first_reverse_flow = self.package_input_message(input_message.data)
            #call the first flow with non blocking call
            self.subflows["first_reverse_flow"].get_reply(
                message_for_first_reverse_flow
            )
        
        #Case where we need to reverse the number for the second time
        elif current_state == "second_reverse_flow":
            
            #Applies a transformation to the input message (renames keys of dictonary so that they match the
            # required format of the second flow)
            message = self.input_interface_second_reverse_flow(input_message)
            
            #Prepares message by editing `reply_data` so that it can get the reply from the first flow
            #back in the input queue of the composite flow 
            message_for_second_reverse_flow = self.package_input_message(input_message.data)
            #call the first flow with non blocking call
            self.subflows["second_reverse_flow"].get_reply(
                message_for_second_reverse_flow
            )
        
        #Case where we need to reply to the initial message (we've already reversed the number twice)
        else:
            message = self.ouput_interface_reply(input_message)
            
            #package ouput message to send back
                #This method packages `response` in a FlowMessage object 
                # containing the necessary metadata to send the message back
                # to the sender of the input message. 
            reply = self.package_output_message(
                input_message = self.flow_state["initial_message"],
                response = message.data
            )
            #send back the reply to initial caller of the flow
            self.send_message(reply)
            
        self.flow_state["current_state"] = self.get_next_state()

The config is identical to the one above (except for `_target_` ) so we won't go through it again.

In [15]:
default_config_reverse_number_sequential_nonblocking = \
{
    "name": "ReverseNumberTwice",
    "description": "A sequential flow that reverses a number twice.",

    # TODO: Define the target
    "_target_": \
        "ReverseNumberFlowModule.ReverseNumberSequentialFlowNonBlocking.ReverseNumberSequentialFlowNonBlocking.instantiate_from_default_config",

    "input_interface": "number",
    "output_interface": "output_number",
    
    "subflows_config": {
        "first_reverse_flow": {
            "_target_": "aiflows.base_flows.AtomicFlow.instantiate_from_default_config",
            "user_id": "local",
            "flow_endpoint": "reverse_number",
            "name": "A proxy flow that calls reverse number to reverse number AGAIN.",
            "description": "A proxy flow that calls reverse number to reverse number.",
        },
        "second_reverse_flow": {
            "_target_": "aiflows.base_flows.AtomicFlow.instantiate_from_default_config",
            "user_id": "local",
            "flow_endpoint": "reverse_number",
            "name": "Proxy Second Reverse",
            "description": "A proxy flow that calls reverse number to reverse number AGAIN.",
        },
    }
}
dict_to_yaml(default_config_reverse_number_sequential_nonblocking, "ReverseNumberFlowModule/ReverseNumberSequentialFlowNonBlocking.yaml")

In [16]:
#Serve the flow (just like before)
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="ReverseNumberFlowModule.ReverseNumberSequentialFlowNonBlocking.ReverseNumberSequentialFlowNonBlocking",
    flow_endpoint="ReverseNumberSequentialFlowNonBlocking",
)

[2024-03-22 00:59:45,625][aiflows.utils.serve_utils:116][INFO] - Started serving ReverseNumberFlowModule.ReverseNumberSequentialFlowNonBlocking.ReverseNumberSequentialFlowNonBlocking at flows:ReverseNumberSequentialFlowNonBlocking.
[2024-03-22 00:59:45,629][aiflows.utils.serve_utils:117][INFO] - dispatch_point: coflows_dispatch
[2024-03-22 00:59:45,630][aiflows.utils.serve_utils:118][INFO] - parallel_dispatch: False
[2024-03-22 00:59:45,632][aiflows.utils.serve_utils:119][INFO] - singleton: False



True

In [17]:

# Get an instance of the flow and run it !!
proxy_reverse_sequential_nonblocking = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="ReverseNumberSequentialFlowNonBlocking", #TODO: SPECIFY THE ENDPOINT
    user_id="local",
    config_overrides=default_config_reverse_number_sequential_nonblocking,
)

input_data = {"id": 0, "number": 12345}

# Package your data in a Flow Message
## Via the package input message method
input_message = proxy_reverse_sequential_nonblocking.package_input_message(input_data)

# Send a message to reverse number and ask to get an answer back in a future

future = proxy_reverse_sequential_nonblocking.get_reply_future(input_message)

# Get the response from the future
#To get the response as a data dictionary
reply_data = future.get_data()
#To get the response as a FlowMessage object
reply_message = future.get_message()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")

[2024-03-22 00:59:46,214][aiflows.utils.serve_utils:543][INFO] - Fetched singleton 40912248-9a21-4766-9c71-f145e3dd78d0
[2024-03-22 00:59:46,381][aiflows.utils.serve_utils:543][INFO] - Fetched singleton 40912248-9a21-4766-9c71-f145e3dd78d0
[2024-03-22 00:59:46,412][aiflows.utils.serve_utils:336][INFO] - Mounted c998880b-9358-4c4c-b378-90fbe89ccf4b at flows:ReverseNumberSequentialFlowNonBlocking:mounts:local:c998880b-9358-4c4c-b378-90fbe89ccf4b
Data sent:
 {'id': 0, 'number': 12345} 

REPLY:
 {'output_number': 12345} 

